In [2]:
import warnings
warnings.simplefilter('ignore')

import gc
import re
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from tqdm.auto import tqdm

In [10]:
df_train_sess = pd.read_csv('./data_for_recstudio/all_task_1_train_sessions.csv')
df_train_sess

,prev_items,next_item,locale
0,['B005ZJTUXE' 'B005ZJTUXE' 'B00P8VIBBG'],B07TVSL9TW,FR
1,['B09M8HSN22' 'B09MTKZNB2' 'B07XWK3G8K' 'B09H7...,B01J5EEEQW,DE
2,['B088ZH3JF1' 'B003YCIEWC' 'B008U4NBE0' 'B073V...,B00TDNLSBU,DE
3,['B01B4S990I' 'B07QJQCHRJ' 'B0723H7YMT'],B09T77G4B5,DE
4,['B079D9HDHP' 'B01E0KS6JC' 'B079D9HDHP'],B079DDND7C,DE
...,...,...,...
3557893,['B004DU3Q9A' 'B09HCCTRPC' 'B09HCCTRPC'],B0165VDN2E,UK
3557894,['B01II34PEC' 'B01II34NC6' 'B01II34PEC' 'B00FV...,B09TF35W1G,IT
3557895,['B0848KJHCK' 'B0B1BRYKX6'],B085PYPS7D,UK
3557896,['B08DJ3NT3L' 'B09FJRQYSV' 'B09J4G565S'],B08K8LLFQ6,UK


In [11]:
df_valid_sess = pd.read_csv('./data_for_recstudio/all_task_1_valid_sessions.csv')
df_valid_sess

,prev_items,next_item,locale
0,['B09VSN9GLS' 'B09VSG9DCG' 'B0BJ5L1ZPH' 'B09VS...,B06XG1LZ6Z,UK
1,['B00390YWXE' 'B00390YWXE' 'B09WM9W6WQ'],B01MSUI4FE,JP
2,['B01BM9V6H8' 'B01MG55XDR' 'B07VYSSRL7'],B01M6625ME,UK
3,['B092ZG24S7' 'B09BNHWWZM' 'B08CB1WG5M' '17880...,0241558573,UK
4,['B0B6NY5RM8' 'B09BJGBBBR'],B09BJF6N8K,JP
...,...,...,...
361576,['B08HH6L4PB' 'B08L8N8HDR'],B00TS5UXGY,UK
361577,['B08X4L1KLZ' 'B09BBX1T4S' 'B09D76FT9D'],B09BCM5NL1,JP
361578,['B0098G6L3M' 'B00ELRLP3O' 'B00PLXGK82' 'B09GS...,B0BC38GHB4,DE
361579,['B07Q2CNLY3' 'B07Q2CNLY3' 'B07BR7DZWN' 'B07Q2...,B08H8SYLMQ,DE


In [4]:
# idx_permu = np.random.permutation(len(df_sess))
# train_num, valid_num = int(len(df_sess) * 0.8), int(len(df_sess) * 0.1)
# test_num = len(df_sess) - train_num - valid_num 
# print(train_num, valid_num, test_num)
# train_idx, valid_idx, test_idx = np.split(idx_permu, [train_num, train_num + valid_num])
# train_idx, valid_idx, test_idx

2884999 360624 360626


(array([1070973, 2705391, 1965079, ...,  944527,  723883,  163305]),
 array([3275574,  575404, 1544217, ..., 3279956, 1506532, 1392533]),
 array([2170579, 2845910, 2934909, ..., 2359272, 1718538, 1953586]))

In [5]:
# df_train_sess = df_sess.loc[train_idx]
# df_valid_sess = df_sess.loc[valid_idx]
# df_test_sess = df_sess.loc[test_idx]
# df_train_sess = df_train_sess.reset_index(drop=True)
# df_valid_sess = df_valid_sess.reset_index(drop=True)
# df_test_sess = df_test_sess.reset_index(drop=True)

In [12]:
df_pred = pd.read_csv('./raw_data/sessions_test_task1.csv')
df_pred

,prev_items,locale
0,['B08V12CT4C' 'B08V1KXBQD' 'B01BVG1XJS' 'B09VC...,DE
1,['B00R9R5ND6' 'B00R9RZ9ZS' 'B00R9RZ9ZS'],DE
2,['B07YSRXJD3' 'B07G7Q5N6G' 'B08C9Q7QVK' 'B07G7...,DE
3,['B08KQBYV43' '3955350843' '3955350843' '39553...,DE
4,['B09FPTCWMC' 'B09FPTQP68' 'B08HMRY8NG' 'B08TB...,DE
...,...,...
316966,['B077SZ2C3Y' 'B0B14M3VZX'],UK
316967,['B08KFHDPY9' 'B0851KTSRZ' 'B08KFHDPY9' 'B0851...,UK
316968,['B07PY1N81F' 'B07Q1Z8SQN' 'B07PY1N81F' 'B07Q1...,UK
316969,['B01MCQMORK' 'B09JYZ325W'],UK


In [13]:
def str2list(x):
    x = x.replace('[', '').replace(']', '').replace("'", '').replace('\n', ' ').replace('\r', ' ')
    l = [i for i in x.split() if i]
    return l

In [16]:
next_item_dict = defaultdict(list)

for _, row in tqdm(df_train_sess.iterrows(), total=len(df_train_sess)):
    prev_items = str2list(row['prev_items'])
    next_item = row['next_item']
    prev_items_length = len(prev_items)
    if prev_items_length <= 1:
        next_item_dict[prev_items[0]].append(next_item)
    else:
        for i, item in enumerate(prev_items[:-1]):
            next_item_dict[item].append(prev_items[i+1])
        next_item_dict[prev_items[-1]].append(next_item)

100%|██████████| 3557898/3557898 [02:39<00:00, 22345.72it/s]


In [18]:
next_item_map = {}

for item in tqdm(next_item_dict):
    counter = Counter(next_item_dict[item])
    next_item_map[item] = [i[0] for i in counter.most_common(100)]

100%|██████████| 1329343/1329343 [00:18<00:00, 72422.99it/s] 


In [19]:
next_item_map

{'B005ZJTUXE': ['B00P8VIBBG',
  'B00AZFJMV8',
  'B005ZJTUXE',
  'B07HJB7911',
  'B016JWSZCW',
  'B07TVSL9TW',
  'B07HJCRPTB',
  'B081S81BQN',
  'B09VYWX9M3',
  'B09C8KVKDR',
  'B00MIKJ4QG',
  'B08LYXMFN6',
  'B00D8WQOB6',
  'B07RF4Q25C',
  'B01MAVSFLI',
  'B0B12YSD85',
  'B0765YSC74',
  'B09Z2NW7H3',
  'B08DP252L6',
  'B09MWCPFBM',
  'B08F9V6TNK',
  'B089TLD66S',
  'B0772R86LG',
  'B01LZQBVFP',
  'B08BJXJHMQ',
  'B014J7P4KU',
  'B06WRT5F1Q',
  'B07JJKRQG1',
  'B0026FC86G'],
 'B00P8VIBBG': ['B00P8VIMTM',
  'B005ZJTUXE',
  'B07TVSL9TW',
  'B014J7P4KU',
  'B0078C1ZP2',
  'B00AZFJMV8',
  'B00P8VIBBG',
  'B00D8WQOB6',
  'B07X5F3DFR',
  'B0744F1Z28',
  'B098BK97TM',
  'B07F1R2JXD',
  'B08JM14F2Z',
  'B012H7QEVC',
  'B095BNKHBY',
  'B09NYH4NDV',
  'B09FXXCCKB',
  'B09C8KVKDR',
  'B08ZSZ4373',
  'B07Z9B3T11',
  'B09BLMDYY8',
  'B09LVTC5PK',
  'B09P9NLKGT',
  'B01IVDV8AI',
  'B013WMBT34',
  'B07MGWFR1J',
  'B00EHD3W3S',
  'B008F7YJQG',
  'B016JWSZCW',
  'B08CHG5PWB'],
 'B09M8HSN22': ['B09MTKZNB

In [20]:
k = []
v = []

for item in next_item_dict:
    k.append(item)
    v.append(next_item_dict[item])
    
df_next = pd.DataFrame({'item': k, 'next_item': v})
df_next = df_next.explode('next_item').reset_index(drop=True)
df_next

,item,next_item
0,B005ZJTUXE,B005ZJTUXE
1,B005ZJTUXE,B00P8VIBBG
2,B005ZJTUXE,B00P8VIBBG
3,B005ZJTUXE,B00P8VIBBG
4,B005ZJTUXE,B07HJCRPTB
...,...,...
15923222,B09KXVZ7YQ,B09KXVDK5F
15923223,B09KXVDK5F,B09KXXGQX8
15923224,B002CWUXO2,B08ZML3GQS
15923225,B00O3BWZJQ,B00O3CISFK


In [21]:
next_item_map['B09W9FND7K']

['B09JSPLN1M',
 'B09W9FND7K',
 'B08LYTWDT6',
 'B07ZF23YBP',
 'B078WW2WN5',
 'B08PPNZXBC']

In [22]:
top200 = df_next['next_item'].value_counts().index.tolist()[:200]
top200

['B07QPV9Z7X',
 'B09NQGVSPD',
 'B0BD5MFPMF',
 'B09VKWKZ16',
 'B08GWS298V',
 'B01N40PO2M',
 'B00NTCH52W',
 'B01MXLEVR7',
 'B01NCX3W3O',
 'B0BDML9477',
 'B09VKWL89R',
 'B0B1MPZWJG',
 'B07N8QY3YH',
 'B0B19YG4MB',
 'B08CN3G4N9',
 'B0BD3XQ9RQ',
 'B0BD88WWQ8',
 'B07RHT52HX',
 'B019GNUT0C',
 'B088FSHMQ3',
 'B001RYNMDA',
 'B09QFPZ9B7',
 'B07XKBLL8F',
 'B09BBX1T4S',
 'B00CWNMV4G',
 'B00NTCHCU2',
 'B0BDZ16ZG4',
 'B09D76FT9D',
 'B014I8SSD0',
 'B098B8PFXY',
 'B08F2NDB39',
 'B0922JX27X',
 'B07WD58H6R',
 'B09QFPYX34',
 'B09HGGV5R5',
 'B0935DN1BN',
 'B07S9FXKZQ',
 'B091V2X68B',
 'B08YKJ17NF',
 'B01B8R6PF2',
 'B0B94K673S',
 'B09BTKNGN5',
 'B07H27J698',
 'B0B61MQD58',
 'B004BIG55Q',
 'B0BCQLVFBM',
 'B0991FVFQJ',
 'B071D8THD2',
 'B09TDL7R7N',
 'B09C7BRP5Y',
 'B08GYKNCCP',
 'B0931VRJT5',
 'B082T6DHB6',
 'B00HZV9WTM',
 'B009Q8492W',
 'B09SWS16W6',
 'B014I8SIJY',
 'B08H93ZRK9',
 'B0B9B1Y8R5',
 'B0BFH812PF',
 'B0BD5NZ6C6',
 'B0B1Q3QML1',
 'B07MLFBJG3',
 'B0B18DBBFB',
 'B07WWY33BK',
 'B07JG9TFSB',
 'B09QFJND

In [23]:
df_valid_sess['last_item'] = df_valid_sess['prev_items'].apply(lambda x: str2list(x)[-1])
df_valid_sess['next_item_prediction'] = df_valid_sess['last_item'].map(next_item_map)
df_valid_sess

,prev_items,next_item,locale,last_item,next_item_prediction
0,['B09VSN9GLS' 'B09VSG9DCG' 'B0BJ5L1ZPH' 'B09VS...,B06XG1LZ6Z,UK,B077XGDMD2,"[B06XGDZVZR, B06XG1LZ6Z, B077XGDMD2, B06XGD9VL..."
1,['B00390YWXE' 'B00390YWXE' 'B09WM9W6WQ'],B01MSUI4FE,JP,B09WM9W6WQ,"[B09LCPT9DQ, B09MRYK5CV, B09WM9W6WQ, B08GZR18S..."
2,['B01BM9V6H8' 'B01MG55XDR' 'B07VYSSRL7'],B01M6625ME,UK,B07VYSSRL7,"[B003TJATC8, B07VYSSRL7, B00L529BAC, B09V5QFBL..."
3,['B092ZG24S7' 'B09BNHWWZM' 'B08CB1WG5M' '17880...,0241558573,UK,1788009975,"[1839941960, 0141376104, 1788009975, 024157563..."
4,['B0B6NY5RM8' 'B09BJGBBBR'],B09BJF6N8K,JP,B09BJGBBBR,"[B0B6PF619D, B09BJF6N8K, B09BJGBBBR, B0B6P6DKN..."
...,...,...,...,...,...
361576,['B08HH6L4PB' 'B08L8N8HDR'],B00TS5UXGY,UK,B08L8N8HDR,"[B08F5D8T22, B01A955L8G, B08HH6L4PB]"
361577,['B08X4L1KLZ' 'B09BBX1T4S' 'B09D76FT9D'],B09BCM5NL1,JP,B09D76FT9D,"[B09BBX1T4S, B09BCM5NL1, B09B9V4PXC, B09XH1YGL..."
361578,['B0098G6L3M' 'B00ELRLP3O' 'B00PLXGK82' 'B09GS...,B0BC38GHB4,DE,B0BC2K5CY1,"[B0BC38GHB4, B07KLCY8NF, B078X1F2JD, B01FE96DM..."
361579,['B07Q2CNLY3' 'B07Q2CNLY3' 'B07BR7DZWN' 'B07Q2...,B08H8SYLMQ,DE,B07Q2CNLY3,"[B08RQDVX71, B08RQR2NPB, B07PY86YPJ, B08H8TLK4..."


In [15]:
# df_test_sess['last_item'] = df_test_sess['prev_items'].apply(lambda x: str2list(x)[-1])
# df_test_sess['next_item_prediction'] = df_test_sess['last_item'].map(next_item_map)
# df_test_sess

,prev_items,next_item,locale,last_item,next_item_prediction
0,['B08GYKF2BH' 'B08H1H4B7P'],B08C7YCWQK,UK,B08H1H4B7P,"[B08C7YCWQK, B08GYKF2BH, B09CNP8LSK, B000TRQGN4]"
1,['B01M62IRCV' 'B01M62IRCV' 'B08HLSXQS9' 'B08DS...,B07259BPG2,UK,B0B1LX67RZ,"[B0B5QGZHR8, B09XC9DV7G, B09SYM81WM, B07259BPG..."
2,['B0B51PL552' 'B07L559LLY' 'B004X56S5Y'],B004X5AZJ4,UK,B004X56S5Y,"[B004X5AZJ4, B09Z3X7DN4]"
3,['B08Z4BGRFV' 'B09T77T7N9'],B09FPQ6NXB,UK,B09T77T7N9,"[B08GPVCWSK, B098KYLL2B, B09F6DQTH8, B0B6GRK25W]"
4,['B07ZYWFFFP' 'B07N9JFGZD' 'B07LCBKPQ2' 'B07LC...,B00B5AN3M6,UK,B09FY639HT,"[B086C6WYTB, B09QG2SDDL]"
...,...,...,...,...,...
360621,['B014R0RUEM' 'B002SWIJ7Y' 'B00QK0KOY6' 'B002S...,B001FMHUNW,JP,B002SWIJ7Y,"[B00QK0KOY6, B014R0RUEM, B00CW9QXG2]"
360622,['B0B9JRSDSX' 'B0B9JQBN63'],B07BB5QRTS,JP,B0B9JQBN63,"[B0B9JRSDSX, B0B9JRMZCY, 484707257X, B0BDKLK1Z..."
360623,['B0B8ND2ZCV' 'B0BHZ6NGR3' 'B0BHZ2GNNR' 'B0B8N...,B09JGFGV3N,UK,B0B8NGDRKZ,"[B0BC888VLM, B0BC8BKBWJ]"
360624,['B09FDV3QDW' 'B09FDV3QDW'],B09TKR9BGM,JP,B09FDV3QDW,"[B0B1J5CX4W, B09FDV3QDW, B0B7S525JP, B07Y3GXQJ..."


In [16]:
# preds = []

# for _, row in tqdm(df_test_sess.iterrows(), total=len(df_test_sess)):
#     pred_orig = row['next_item_prediction']
#     pred = pred_orig
#     prev_items = str2list(row['prev_items'])
#     if type(pred) == float:
#         pred = top200[:100]
#     else:
#         if len(pred_orig) < 100:
#             for i in top200:
#                 if i not in pred_orig and i not in prev_items:
#                     pred.append(i)
#                 if len(pred) >= 100:
#                     break
#         else:
#             pred = pred[:100]
#     preds.append(pred)

100%|██████████| 360626/360626 [00:41<00:00, 8708.18it/s] 


In [17]:
# df_test_sess['next_item_prediction'] = preds
# df_test_sess

,prev_items,next_item,locale,last_item,next_item_prediction
0,['B08GYKF2BH' 'B08H1H4B7P'],B08C7YCWQK,UK,B08H1H4B7P,"[B08C7YCWQK, B08GYKF2BH, B09CNP8LSK, B000TRQGN..."
1,['B01M62IRCV' 'B01M62IRCV' 'B08HLSXQS9' 'B08DS...,B07259BPG2,UK,B0B1LX67RZ,"[B0B5QGZHR8, B09XC9DV7G, B09SYM81WM, B07259BPG..."
2,['B0B51PL552' 'B07L559LLY' 'B004X56S5Y'],B004X5AZJ4,UK,B004X56S5Y,"[B004X5AZJ4, B09Z3X7DN4, B07QPV9Z7X, B09NQGVSP..."
3,['B08Z4BGRFV' 'B09T77T7N9'],B09FPQ6NXB,UK,B09T77T7N9,"[B08GPVCWSK, B098KYLL2B, B09F6DQTH8, B0B6GRK25..."
4,['B07ZYWFFFP' 'B07N9JFGZD' 'B07LCBKPQ2' 'B07LC...,B00B5AN3M6,UK,B09FY639HT,"[B086C6WYTB, B09QG2SDDL, B07QPV9Z7X, B09NQGVSP..."
...,...,...,...,...,...
360621,['B014R0RUEM' 'B002SWIJ7Y' 'B00QK0KOY6' 'B002S...,B001FMHUNW,JP,B002SWIJ7Y,"[B00QK0KOY6, B014R0RUEM, B00CW9QXG2, B07QPV9Z7..."
360622,['B0B9JRSDSX' 'B0B9JQBN63'],B07BB5QRTS,JP,B0B9JQBN63,"[B0B9JRSDSX, B0B9JRMZCY, 484707257X, B0BDKLK1Z..."
360623,['B0B8ND2ZCV' 'B0BHZ6NGR3' 'B0BHZ2GNNR' 'B0B8N...,B09JGFGV3N,UK,B0B8NGDRKZ,"[B0BC888VLM, B0BC8BKBWJ, B07QPV9Z7X, B09NQGVSP..."
360624,['B09FDV3QDW' 'B09FDV3QDW'],B09TKR9BGM,JP,B09FDV3QDW,"[B0B1J5CX4W, B09FDV3QDW, B0B7S525JP, B07Y3GXQJ..."


In [24]:
preds = []

for _, row in tqdm(df_valid_sess.iterrows(), total=len(df_valid_sess)):
    pred_orig = row['next_item_prediction']
    pred = pred_orig
    prev_items = str2list(row['prev_items'])
    if type(pred) == float:
        pred = top200[:100]
    else:
        if len(pred_orig) < 100:
            for i in top200:
                if i not in pred_orig and i not in prev_items:
                    pred.append(i)
                if len(pred) >= 100:
                    break
        else:
            pred = pred[:100]
    preds.append(pred)

100%|██████████| 361581/361581 [00:40<00:00, 8993.87it/s] 


In [25]:
df_valid_sess['next_item_prediction'] = preds
df_valid_sess

,prev_items,next_item,locale,last_item,next_item_prediction
0,['B09VSN9GLS' 'B09VSG9DCG' 'B0BJ5L1ZPH' 'B09VS...,B06XG1LZ6Z,UK,B077XGDMD2,"[B06XGDZVZR, B06XG1LZ6Z, B077XGDMD2, B06XGD9VL..."
1,['B00390YWXE' 'B00390YWXE' 'B09WM9W6WQ'],B01MSUI4FE,JP,B09WM9W6WQ,"[B09LCPT9DQ, B09MRYK5CV, B09WM9W6WQ, B08GZR18S..."
2,['B01BM9V6H8' 'B01MG55XDR' 'B07VYSSRL7'],B01M6625ME,UK,B07VYSSRL7,"[B003TJATC8, B07VYSSRL7, B00L529BAC, B09V5QFBL..."
3,['B092ZG24S7' 'B09BNHWWZM' 'B08CB1WG5M' '17880...,0241558573,UK,1788009975,"[1839941960, 0141376104, 1788009975, 024157563..."
4,['B0B6NY5RM8' 'B09BJGBBBR'],B09BJF6N8K,JP,B09BJGBBBR,"[B0B6PF619D, B09BJF6N8K, B09BJGBBBR, B0B6P6DKN..."
...,...,...,...,...,...
361576,['B08HH6L4PB' 'B08L8N8HDR'],B00TS5UXGY,UK,B08L8N8HDR,"[B08F5D8T22, B01A955L8G, B08HH6L4PB, B07QPV9Z7..."
361577,['B08X4L1KLZ' 'B09BBX1T4S' 'B09D76FT9D'],B09BCM5NL1,JP,B09D76FT9D,"[B09BBX1T4S, B09BCM5NL1, B09B9V4PXC, B09XH1YGL..."
361578,['B0098G6L3M' 'B00ELRLP3O' 'B00PLXGK82' 'B09GS...,B0BC38GHB4,DE,B0BC2K5CY1,"[B0BC38GHB4, B07KLCY8NF, B078X1F2JD, B01FE96DM..."
361579,['B07Q2CNLY3' 'B07Q2CNLY3' 'B07BR7DZWN' 'B07Q2...,B08H8SYLMQ,DE,B07Q2CNLY3,"[B08RQDVX71, B08RQR2NPB, B07PY86YPJ, B08H8TLK4..."


In [26]:
df_valid_sess.iloc[14]

prev_items                                                 ['B001DQA7GC']
next_item                                                      B09B2XXBFR
locale                                                                 UK
last_item                                                      B001DQA7GC
next_item_prediction    [B001DQA7GC, B07VRWWFJP, B00O9RYS7Q, B07PYXZHS...
Name: 14, dtype: object

In [27]:
# mrr for valid
mrr_valid = []
for i in tqdm(range(len(df_valid_sess))):
    mrr = 0.0
    ground_truth = df_valid_sess.iloc[i]['next_item']
    for i, item in enumerate(df_valid_sess.iloc[i]['next_item_prediction']):
        if item == ground_truth:
            mrr = 1.0 / (i + 1)
            break
    mrr_valid.append(mrr)
mrr_valid = np.array(mrr_valid).mean()
mrr_valid

100%|██████████| 361581/361581 [00:28<00:00, 12578.76it/s]


0.24878226984087287

In [29]:
# mrr for test
# mrr_test = []
# for i in tqdm(range(len(df_test_sess))):
#     mrr = 0.0
#     ground_truth = df_test_sess.iloc[i]['next_item']
#     for i, item in enumerate(df_test_sess.iloc[i]['next_item_prediction']):
#         if item == ground_truth:
#             mrr = 1.0 / (i + 1)
#             break
#     mrr_test.append(mrr)
# mrr_test = np.array(mrr_test).mean()
# mrr_test

100%|██████████| 360626/360626 [00:33<00:00, 10914.50it/s]


0.24074462120638251

In [28]:
df_pred['last_item'] = df_pred['prev_items'].apply(lambda x: str2list(x)[-1])
df_pred['next_item_prediction'] = df_pred['last_item'].map(next_item_map)
df_pred

,prev_items,locale,last_item,next_item_prediction
0,['B08V12CT4C' 'B08V1KXBQD' 'B01BVG1XJS' 'B09VC...,DE,B099NQFMG7,"[B099NS1XPG, B08496TCCQ, B01BVG1XJS, B099NR3X6..."
1,['B00R9R5ND6' 'B00R9RZ9ZS' 'B00R9RZ9ZS'],DE,B00R9RZ9ZS,"[B004ZXMV4Q, B00R9R5ND6, B095TQTZXY, B001LEO21..."
2,['B07YSRXJD3' 'B07G7Q5N6G' 'B08C9Q7QVK' 'B07G7...,DE,B07G7Q5N6G,"[B08C9Q7QVK, B07G7Q5N6G, B07YSRXJD3, B0B5QNFWJ..."
3,['B08KQBYV43' '3955350843' '3955350843' '39553...,DE,3955350843,"[395535086X, 3955350843, 3772476953, B0829LZFT..."
4,['B09FPTCWMC' 'B09FPTQP68' 'B08HMRY8NG' 'B08TB...,DE,B09J945WQR,"[B09J8V18FL, B09J8T6TTH, B09J8SKX9G, B09J8V9RQ..."
...,...,...,...,...
316966,['B077SZ2C3Y' 'B0B14M3VZX'],UK,B0B14M3VZX,"[B08X9L5RGD, B09Y4HKGKT, B07V5FL8G6, B09MW64JGM]"
316967,['B08KFHDPY9' 'B0851KTSRZ' 'B08KFHDPY9' 'B0851...,UK,B081YDH55K,"[B0989BHLSY, B09CPNS7XV, B09895QPQF, B09CPP92Q..."
316968,['B07PY1N81F' 'B07Q1Z8SQN' 'B07PY1N81F' 'B07Q1...,UK,B09HL11V5B,"[B09HKZBNZH, B09HZSRJWW, B09HX9VGW4, B09HL141Q..."
316969,['B01MCQMORK' 'B09JYZ325W'],UK,B09JYZ325W,"[B07TR5LQSL, B09JYZ325W, B08FB464L7, B08JG8TSC..."


In [30]:
preds = []

for _, row in tqdm(df_pred.iterrows(), total=len(df_pred)):
    pred_orig = row['next_item_prediction']
    pred = pred_orig
    prev_items = str2list(row['prev_items'])
    if type(pred) == float:
        pred = top200[:100]
    else:
        if len(pred_orig) < 100:
            for i in top200:
                if i not in pred_orig and i not in prev_items:
                    pred.append(i)
                if len(pred) >= 100:
                    break
        else:
            pred = pred[:100]
    preds.append(pred)

100%|██████████| 316971/316971 [00:16<00:00, 19323.77it/s]


In [31]:
df_pred['next_item_prediction'] = preds
df_pred

,prev_items,locale,last_item,next_item_prediction
0,['B08V12CT4C' 'B08V1KXBQD' 'B01BVG1XJS' 'B09VC...,DE,B099NQFMG7,"[B099NS1XPG, B08496TCCQ, B01BVG1XJS, B099NR3X6..."
1,['B00R9R5ND6' 'B00R9RZ9ZS' 'B00R9RZ9ZS'],DE,B00R9RZ9ZS,"[B004ZXMV4Q, B00R9R5ND6, B095TQTZXY, B001LEO21..."
2,['B07YSRXJD3' 'B07G7Q5N6G' 'B08C9Q7QVK' 'B07G7...,DE,B07G7Q5N6G,"[B08C9Q7QVK, B07G7Q5N6G, B07YSRXJD3, B0B5QNFWJ..."
3,['B08KQBYV43' '3955350843' '3955350843' '39553...,DE,3955350843,"[395535086X, 3955350843, 3772476953, B0829LZFT..."
4,['B09FPTCWMC' 'B09FPTQP68' 'B08HMRY8NG' 'B08TB...,DE,B09J945WQR,"[B09J8V18FL, B09J8T6TTH, B09J8SKX9G, B09J8V9RQ..."
...,...,...,...,...
316966,['B077SZ2C3Y' 'B0B14M3VZX'],UK,B0B14M3VZX,"[B08X9L5RGD, B09Y4HKGKT, B07V5FL8G6, B09MW64JG..."
316967,['B08KFHDPY9' 'B0851KTSRZ' 'B08KFHDPY9' 'B0851...,UK,B081YDH55K,"[B0989BHLSY, B09CPNS7XV, B09895QPQF, B09CPP92Q..."
316968,['B07PY1N81F' 'B07Q1Z8SQN' 'B07PY1N81F' 'B07Q1...,UK,B09HL11V5B,"[B09HKZBNZH, B09HZSRJWW, B09HX9VGW4, B09HL141Q..."
316969,['B01MCQMORK' 'B09JYZ325W'],UK,B09JYZ325W,"[B07TR5LQSL, B09JYZ325W, B08FB464L7, B08JG8TSC..."


In [32]:
df_pred['next_item_prediction'].apply(len).describe()

count    316971.0
mean        100.0
std           0.0
min         100.0
25%         100.0
50%         100.0
75%         100.0
max         100.0
Name: next_item_prediction, dtype: float64

In [35]:
df_pred[['locale', 'next_item_prediction']].to_parquet('submission_task1_new_split.parquet', engine='pyarrow')

In [3]:
df_my_pred = pd.read_parquet("/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/predictions/SASRec_Next/kdd_cup_2023/2023-03-28-01-44-20.parquet")
df_my_pred.head(5)

,locale,next_item_prediction
0,DE,"[B0B6BHYW9N, B092D6TVGH, B0B139RW78, B09Z16NCJ..."
1,DE,"[B094P14PNJ, B08R5KGSXJ, B08Z3DG88N, B08PVJNH6..."
2,DE,"[B07Q329STR, B07V6QXGHX, B07JN5LBS7, B09QFWT3G..."
3,DE,"[B077PBQF2X, B07RKWV54M, B07BR4X8L6, B07X43718..."
4,DE,"[B0BGMHNNGQ, B09CXDNFRC, B08WZ2F7K9, B08PT4XZF..."


In [5]:
df_my_pred2 = pd.read_parquet("/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/predictions/SASRec2/kdd_cup_2023/2023-03-27-20-06-42.parquet")
df_my_pred2.head(5)

,locale,next_item_prediction
0,DE,"[B099NS1XPG, B07LG5T3V9, B099NR3X6D, B086CJF45..."
1,DE,"[B004ZXMV4Q, B095TQTZXY, B07WDK8PPN, B08WC5CF5..."
2,DE,"[B000KSLHNQ, B0BJ7H1822, B0B6MZZ674, B0B5QNFWJ..."
3,DE,"[395535086X, B089K81TDV, B09KNCTB5F, 377247695..."
4,DE,"[B09J8TWRV3, B09J8SKX9G, B089CVQ2FS, B07JGZH51..."


In [13]:
df_my_pred.iloc[123456]

locale                                                                 JP
next_item_prediction    [B09HKQQBCH, B096ZQNMHH, B07Q73VM9Y, B099K6PP3...
Name: 123456, dtype: object

In [12]:
df_my_pred2.iloc[123456]

locale                                                                 JP
next_item_prediction    [B00A6LHPZ6, B000MU34Y2, B000MU4YKA, B00AXUC3M...
Name: 123456, dtype: object

In [4]:
df_my_pred['next_item_prediction'].apply(len).describe()

count    316971.0
mean        100.0
std           0.0
min         100.0
25%         100.0
50%         100.0
75%         100.0
max         100.0
Name: next_item_prediction, dtype: float64

In [41]:
df_pred[['locale', 'next_item_prediction']].iloc[0]['next_item_prediction']

['B099NS1XPG',
 'B099NR3X6D',
 'B08496TCCQ',
 'B09WDSH4CD',
 'B07QPV9Z7X',
 'B09NQGVSPD',
 'B0BD5MFPMF',
 'B01NCX3W3O',
 'B00NTCH52W',
 'B01N40PO2M',
 'B0B1MPZWJG',
 'B08GWS298V',
 'B09VKWKZ16',
 'B07XKBLL8F',
 'B01MXLEVR7',
 'B07RHT52HX',
 'B0BD3XQ9RQ',
 'B0BDML9477',
 'B07N8QY3YH',
 'B0BD88WWQ8',
 'B09VKWL89R',
 'B088FSHMQ3',
 'B08CN3G4N9',
 'B091V2X68B',
 'B0922JX27X',
 'B0BDZ16ZG4',
 'B0B19YG4MB',
 'B019GNUT0C',
 'B07S9FXKZQ',
 'B09D76FT9D',
 'B001RYNMDA',
 'B08YKJ17NF',
 'B00NTCHCU2',
 'B07WD58H6R',
 'B09BBX1T4S',
 'B00CWNMV4G',
 'B0935DN1BN',
 'B09QFPZ9B7',
 'B09HGGV5R5',
 'B004BIG55Q',
 'B014I8SSD0',
 'B0B18DBBFB',
 'B098B8PFXY',
 'B0991FVFQJ',
 'B08GYKNCCP',
 'B09BTKNGN5',
 'B08H93ZRK9',
 'B09QFPYX34',
 'B09SWS16W6',
 'B01B8R6PF2',
 'B08F2NDB39',
 'B07H27J698',
 'B0B18CK9DV',
 'B014I8SIJY',
 'B0BCQLVFBM',
 'B0931VRJT5',
 'B01KNWYGMW',
 'B07MLFBJG3',
 'B0B1Q3QML1',
 'B00HZV9WTM',
 'B08ZMQT158',
 'B077B9X3SB',
 'B0B9B1Y8R5',
 'B08LSNJQ1N',
 'B07XXZP2CK',
 'B07PNL5STG',
 'B009Q849

In [42]:
df_my_pred['next_item_prediction'].iloc[0]

array(['B08L9CZ7BW', 'B095C349SJ', 'B07PHQ22FY', 'B099XL3VS4',
       'B08X12TS8R', 'B09Z16NCJT', 'B0B2X3LC93', 'B08DY7S8DR',
       'B08HDCY9VR', 'B0BCGXM4WH', 'B0BGJDSD38', 'B085QYQ112',
       'B0BDFTFPZ3', 'B088GNP6TB', 'B09F3P3DQD', 'B0851LN8QG',
       'B0BH8NYPV6', 'B0BL768TBK', 'B08FCZPSH2', 'B07Y22TWNF',
       '3735880266', 'B08CY2WJFF', 'B0B7R1SVSG', 'B09YPR8BBJ',
       'B08Q7SX3V6', 'B0BDM9JYGJ', 'B096HSYFL2', 'B0B63D692W',
       'B07XZF3JFZ', 'B08X18GCYD', 'B0B7CBD6M6', 'B08CS1WFN8',
       'B08QW43S2L', 'B07CZ4DLCP', 'B09P4B2VS2', 'B08151FJRJ',
       'B0851MXXR3', 'B09FPV7NF8', 'B00F5UX7EQ', 'B095CSPRRF',
       'B093SYL61B', 'B09DDDR9WV', 'B0B3264C64', 'B07H2BZ5TP',
       'B0B9XTSD8Z', 'B08X23BH9M', 'B085QZ2FJT', 'B085QZ6FLD',
       'B08Q7HL9QV', 'B0B5CYGH1B', 'B0BD5JBWKW', 'B0B6ZPHNHQ',
       'B07GQNC5KQ', 'B099MWSH7B', 'B00FFTDZTY', '3473491764',
       'B09NLCC7TD', 'B0BCGV6372', 'B0BHJKVMB4', 'B09P4BKWPL',
       'B09JCH4D5B', 'B08X16LL57', 'B09JRGQ336', 'B095C